# Задание 1

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [2]:
import pandas as pd

In [3]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")

Установка библиотек

In [4]:
pip install --upgrade gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install razdel

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=1f8b0f22e116f5a1f451c233364a24c6918a66f206c4af43294f705685d744d9
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\72\b0\3f\1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [7]:
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from razdel import tokenize  # https://github.com/natasha/razdel
import pymorphy2

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [10]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


Wall time: 46.2 s


In [13]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 7min 9s


In [14]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [15]:
print([common_dictionary[i] for i in range(10, 15)])

['ватутин', 'взаимодействие', 'власть', 'войти', 'вячеслав']


In [16]:
from gensim.models import LdaModel

In [17]:

%%time
from gensim.models import LdaModel

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary, passes=5)

Wall time: 7min 6s


In [18]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [19]:

# Create a new corpus, made of previously unseen documents.
# Создайте новый корпус, состоящий из ранее невидимых документов.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]

print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(5, 0.3331048),
 (7, 0.38534585),
 (9, 0.022027293),
 (11, 0.1898032),
 (15, 0.02406676),
 (21, 0.029088639)]

In [20]:

x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: германия северный европа белоруссия польша япония японский
topic_1: год млрд млн рубль компания рост страна
topic_2: конкурс супруг детский ресторан акционер майкл победитель
topic_3: мозг врач пациент университет лечение проверка фестиваль
topic_4: вино альянс клуб испания снизить сосед испанский
topic_5: это всё человек который мочь весь свой
topic_6: nn президент фонд путин глава россия владимир
topic_7: планета nn налог место мир стать первый
topic_8: исследование станция восток штат век прогнозировать дальний
topic_9: поверхность компьютерный горный багаж тверской метан юг
topic_10: банк это мочь санкция снижение россия который
topic_11: год который это также время город новый
topic_12: россия российский совет наука nn александр комитет
topic_13: газ район погибнуть пострадать авария результат огонь
topic_14: ракета запуск блок парламент старт запустить км
topic_15: день температура школа москва рейс фильм праздник
topic_16: сша россия украина российский страна это украин

Давайте напишем функцию, которая будет нам возвращать векторное представление новости/

In [21]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(
        zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [22]:

topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)


,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.013326,0.0,0.0,0.0,0.019519,0.000000,0.139715,0.115162,0.0,...,0.000000,0.228218,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,4896,0.000000,0.0,0.0,0.0,0.000000,0.110153,0.000000,0.357504,0.0,...,0.000000,0.000000,0.092405,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.184197
2,4897,0.000000,0.0,0.0,0.0,0.000000,0.333157,0.000000,0.385344,0.0,...,0.024067,0.000000,0.000000,0.0,0.0,0.000000,0.029096,0.000000,0.0,0.000000
3,4898,0.000000,0.0,0.0,0.0,0.050418,0.424892,0.000000,0.266145,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.086529,0.000000,0.063272,0.0,0.000000
4,4899,0.000000,0.0,0.0,0.0,0.116002,0.000000,0.093841,0.028203,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.280113,0.000000,0.000000,0.0,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

In [23]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [24]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [25]:
doc_dict[293622]

array([0.        , 0.        , 0.04452455, 0.        , 0.        ,
       0.0718068 , 0.15829754, 0.19217636, 0.01559653, 0.        ,
       0.0409411 , 0.13041152, 0.0385638 , 0.        , 0.        ,
       0.        , 0.        , 0.04023201, 0.1159236 , 0.        ,
       0.        , 0.        , 0.03954242, 0.08839045, 0.01629931])

In [27]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector
def get_user_embedding_med(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector


In [28]:
def score_with_user_embedding(func):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: func(x), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] + [f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(
    X[[f'topic_{i}' for i in range(25)]], X['churn'], random_state=0)
    
    logreg = LogisticRegression()

    #обучим наш пайплайн
    logreg.fit(X_train, y_train)

    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]

    # Рассчитаем Precision, Recall, F_score, roc_auc_score
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc_score = roc_auc_score(y_test, preds)

    return round(thresholds[ix], 4), round(fscore[ix], 4), round(precision[ix], 4), round(recall[ix], 4), round(roc_auc_score, 4)

In [29]:
target = pd.read_csv("users_churn.csv")

Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [31]:
results = pd.DataFrame(np.array([
    score_with_user_embedding(func=get_user_embedding_mean),
    score_with_user_embedding(func=get_user_embedding_med),
    score_with_user_embedding(func=get_user_embedding_max)
]), columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['func'] = ['mean', 'median', 'max']
results = results.set_index('func')

In [32]:
results

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
func,,,,,
mean,0.3023,0.7746,0.7336,0.8204,0.9736
median,0.2670,0.7593,0.6949,0.8367,0.9690
max,0.2896,0.7940,0.7336,0.8653,0.9757


Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

MAX показал себя лучше остальных
Скорее всего разброс в интесесах пользователя становится меньше, интересы как-бы более узкие, но более точные.